In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [3]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders
from utils.unrolled_rnn import make_train_op
from utils.batcher import generate_epoch

In [4]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [18]:
EMBEDDING_SIZE=64
HIDDEN_SIZE=256
BATCH_SIZE=32
NUM_STEPS=16
NUM_EPOCHS_INIT_LR=3
NUM_EPOCHS_TOTAL=8
INITIAL_LR=5e0
LR_DECAY_RATE=0.75
MAX_NORM=0.5

In [19]:
tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=vocab_size,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)
training_nodes = make_train_op(
    summary_nodes['loss'],
    placeholders['learning_rate'],
    placeholders['max_norm'],
)

In [20]:
training_outputs = {**summary_nodes, **training_nodes}
with tf.Session() as sess:

    # Bookkeeping
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    learning_rate = INITIAL_LR
    max_norm = MAX_NORM
    for i in range(NUM_EPOCHS_TOTAL):
        if i >= NUM_EPOCHS_INIT_LR:
            learning_rate *= LR_DECAY_RATE
        for batch_idx, (inputs, targets) in enumerate(generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)):
            outputs = sess.run(
                training_outputs,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets,
                    placeholders['learning_rate']: learning_rate,
                    placeholders['max_norm']: max_norm,
                }
            )
            if (batch_idx % 64 == 63):
                print('step: {0}    loss: {1}    gradient norm: {2}     correct words: {3}'.format(
                    batch_idx+1,
                    outputs['loss'],
                    outputs['gradient_global_norm'],
                    outputs['num_correct_predictions'],
                ))
                
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_val, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('validation perplexity:', np.exp(total_loss / total_batches))
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_test, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('test perplexity:', np.exp(total_loss / total_batches))

    # Bookkeeping        
    writer.close()
    coord.request_stop()
    coord.join(threads)

step: 64    loss: 7.71875    gradient norm: 2.517578125     correct words: 22
step: 128    loss: 9.6171875    gradient norm: 5.125     correct words: 29
step: 192    loss: 6.8828125    gradient norm: 0.325439453125     correct words: 45
step: 256    loss: 6.7265625    gradient norm: 0.53271484375     correct words: 36
step: 320    loss: 6.60546875    gradient norm: 0.4287109375     correct words: 28
step: 384    loss: 6.640625    gradient norm: 0.49853515625     correct words: 43
step: 448    loss: 6.71484375    gradient norm: 0.6025390625     correct words: 22
step: 512    loss: 6.640625    gradient norm: 0.484130859375     correct words: 33
step: 576    loss: 6.9921875    gradient norm: 1.7861328125     correct words: 72
step: 640    loss: 6.4296875    gradient norm: 0.650390625     correct words: 28
step: 704    loss: 6.2109375    gradient norm: 0.48046875     correct words: 46
step: 768    loss: 6.4375    gradient norm: 0.5185546875     correct words: 45
step: 832    loss: 6.394531

step: 960    loss: 4.88671875    gradient norm: 0.58984375     correct words: 115
step: 1024    loss: 5.65625    gradient norm: 1.634765625     correct words: 97
step: 1088    loss: 5.08203125    gradient norm: 0.853515625     correct words: 111
step: 1152    loss: 5.1640625    gradient norm: 0.5244140625     correct words: 111
step: 1216    loss: 4.734375    gradient norm: 0.50537109375     correct words: 114
step: 1280    loss: 4.8046875    gradient norm: 0.578125     correct words: 114
step: 1344    loss: 4.953125    gradient norm: 0.56591796875     correct words: 101
step: 1408    loss: 4.796875    gradient norm: 0.53955078125     correct words: 96
step: 1472    loss: 5.140625    gradient norm: 0.615234375     correct words: 108
step: 1536    loss: 5.1484375    gradient norm: 0.8564453125     correct words: 103
step: 1600    loss: 4.9921875    gradient norm: 0.5185546875     correct words: 115
step: 1664    loss: 4.953125    gradient norm: 0.51806640625     correct words: 109
step:

step: 1728    loss: 4.59375    gradient norm: 0.58447265625     correct words: 120
step: 1792    loss: 4.48828125    gradient norm: 0.5771484375     correct words: 119
validation perplexity: 149.759878754
test perplexity: 140.844472546
step: 64    loss: 4.6484375    gradient norm: 0.56396484375     correct words: 119
step: 128    loss: 4.8359375    gradient norm: 0.62744140625     correct words: 115
step: 192    loss: 4.53515625    gradient norm: 0.59375     correct words: 124
step: 256    loss: 4.51171875    gradient norm: 0.583984375     correct words: 130
step: 320    loss: 4.4140625    gradient norm: 0.57666015625     correct words: 131
step: 384    loss: 4.59375    gradient norm: 0.55419921875     correct words: 114
step: 448    loss: 4.82421875    gradient norm: 0.6357421875     correct words: 111
step: 512    loss: 4.640625    gradient norm: 0.57470703125     correct words: 126
step: 576    loss: 4.46875    gradient norm: 0.5927734375     correct words: 132
step: 640    loss: 4.